In [ ]:
#The code presented in this section is used to take the names of the founders in a spreadsheet cell and
#obtain their LinkedIn profiles by using the Python library Requests to perform a search and the package
#Beautiful Soup to scrape the results.

import csv
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

# open excel file and read profiles
df = pd.read_excel('list_founders.xlsx')

# find linkedin address from profile name and company
profile_urls = []  # To store the Profile URLs
companies = df.iloc[:, 0].tolist()  # create list of companies
founders = df.iloc[:, 2].tolist()  # create list of founders
founders = [element for element in founders if str(element) != 'nan']
result = []

for company, founder in zip(companies, founders):
    founder_names = []
    if ', and' in founder:
        names = founder.split(', and')
        for name in names:
            name = name.strip()
            if ',' in name:
                founder_names.extend([n.strip() for n in name.split(',')])
            elif 'and' in name:
                founder_names.extend([n.strip() for n in name.split('and')])
            else:
                founder_names.append(name)
    else:
        founder_names = [name.strip() for name in re.split(r',\s| and ', founder)]
    result.append([company] + founder_names)

for i in result:
    company = i[0]
    names = pd.Series(i[1:])
    names = names[~pd.isna(names)].values.tolist()
    for name in names:
        profile = []
        text = 'site:linkedin.com/in/ AND "{}" AND "{}" '.format(company, name)
        query = 'https://google.com/search?q=' + text
        cookies = {"CONSENT": "YES+shp.gws-20210330-0-RC1.de+FX+412"}
        response = requests.get(query, cookies=cookies)
        soup = BeautifulSoup(response.text, 'html.parser')
        for anchor in soup.find_all('a'):
            url = anchor["href"]
            if 'https://www.linkedin.com/' in url:
                url = url[7:url.find('&')]
                profile.append(url)
        try:
            profile_urls.append(profile[0])
        except:
            continue

print(profile_urls)
